## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,77.09,81,18,19.73,light rain
1,1,Saskylakh,RU,71.9167,114.0833,59.70,64,100,5.14,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,60.96,100,75,0.00,broken clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,61.84,84,100,17.83,overcast clouds
4,4,Nantucket,US,41.2835,-70.0995,83.25,70,75,5.75,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Nantucket,US,41.2835,-70.0995,83.25,70,75,5.75,broken clouds
9,9,Wilmington,US,34.2257,-77.9447,88.95,71,75,9.22,broken clouds
12,12,Banjul,GM,13.4500,-16.5833,82.72,83,75,8.05,broken clouds
27,27,Jamestown,US,42.0970,-79.2353,85.17,58,1,3.00,clear sky
29,29,Taft,IR,31.7471,54.2088,87.75,13,20,6.91,few clouds
35,35,Hilo,US,19.7297,-155.0900,81.25,73,90,4.61,overcast clouds
36,36,Bodden Town,KY,19.2833,-81.2500,89.98,71,40,8.99,scattered clouds
37,37,Bethel,US,41.3712,-73.4140,84.47,70,1,8.05,light rain
40,40,Alenquer,BR,-1.9417,-54.7383,88.14,38,100,5.01,overcast clouds
47,47,Hithadhoo,MV,-0.6000,73.0833,80.71,76,100,4.38,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                136
City                   136
Country                136
Lat                    136
Lng                    136
Max Temp               136
Humidity               136
Cloudiness             136
Wind Speed             136
Current Description    136
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Nantucket,US,83.25,broken clouds,41.2835,-70.0995,
9,Wilmington,US,88.95,broken clouds,34.2257,-77.9447,
12,Banjul,GM,82.72,broken clouds,13.4500,-16.5833,
27,Jamestown,US,85.17,clear sky,42.0970,-79.2353,
29,Taft,IR,87.75,few clouds,31.7471,54.2088,
35,Hilo,US,81.25,overcast clouds,19.7297,-155.0900,
36,Bodden Town,KY,89.98,scattered clouds,19.2833,-81.2500,
37,Bethel,US,84.47,light rain,41.3712,-73.4140,
40,Alenquer,BR,88.14,overcast clouds,-1.9417,-54.7383,
47,Hithadhoo,MV,80.71,overcast clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
#drop rows that don't have hotel names
hotel_df = hotel_df.replace({'': np.nan})
clean_hotel_df = hotel_df.dropna(axis=0, how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Nantucket,US,83.25,broken clouds,41.2835,-70.0995,Jared Coffin House
9,Wilmington,US,88.95,broken clouds,34.2257,-77.9447,"Hotel Ballast Wilmington, Tapestry Collection ..."
12,Banjul,GM,82.72,broken clouds,13.4500,-16.5833,Laico Atlantic Banjul
27,Jamestown,US,85.17,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
29,Taft,IR,87.75,few clouds,31.7471,54.2088,بوم گردی حاجی زینل تفت
...,...,...,...,...,...,...,...
628,Nanakuli,US,82.35,broken clouds,21.3906,-158.1547,Camp Pālehua
635,Pochutla,MX,87.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
649,Ottumwa,US,86.68,broken clouds,41.0200,-92.4113,Hampton Inn Ottumwa
658,Misratah,LY,81.90,clear sky,32.3754,15.0925,فندق الماسة


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))